In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time
import re

In [10]:
# 크롬을 열고 전체화면으로 전환 인프런 사이트를 엽니다.
driver = webdriver.Chrome()
driver.maximize_window()

In [3]:
# 추출할 데이터를 담을 리스트를 생성합니다.
want_list = ['it-programming', 'game-dev-all','data-science', 'it', 'business/task-automation', 'hardware']
list_data = []

In [9]:
# 개발/프로그래밍

def inflearn_crawling(x):
    page = 1
    for j in range(1,10000):
        title1=driver.find_element(
            By.XPATH, f'//*[@id="courses_section"]/div/div/div/main/div[4]/div/div[1]/div/a/div[2]/div[1]').text
        driver.get(f'https://www.inflearn.com/courses/{x}?order=seq&page={j}')
        time.sleep(1)
        
        title2=driver.find_element(
            By.XPATH, f'//*[@id="courses_section"]/div/div/div/main/div[4]/div/div[1]/div/a/div[2]/div[1]').text
        
        if page >=2:
            if title1 == title2:
                break
            else:
                pass
            
        page +=1
        

        for i in range(1,25):
                try:
                    try:
                        driver.find_element(By.XPATH, f'//*[@id="courses_section"]/div/div/div/main/div[4]/div/div[{i}]/div/a/div[2]/span').is_displayed()
                    except:
                        # 가격
                        class_price = driver.find_element(By.XPATH, f'//*[@id="courses_section"]/div/div/div/main/div[4]/div/div[{i}]/div/a/div[2]/div[4]').text

                        # 항목 클릭
                        try :
                            element = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="courses_section"]/div/div/div/main/div[4]/div/div[{i}]/div/a/div[1]/figure/img')))
                            driver.execute_script("arguments[0].click();", element)                          
                        except :
                            element = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="courses_section"]/div/div/div/main/div[4]/div/div[{i}]/div/a/div[1]/section/video')))
                            driver.execute_script("arguments[0].click();", element)

                        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/section/div[1]/div[1]/div/div/div[2]/div[3]/span[2]/strong')))
                        
                        try:
                            driver.find_element(By.XPATH, '//*[@id="main"]/section/div[1]/div[1]/div/div/div[2]/div[2]/h1').is_displayed()
                        except:
                            driver.refresh()
                            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/section/div[1]/div[1]/div/div/div[2]/div[3]/span[2]/strong')))
                        # 강좌명 
                        class_name = driver.find_element(By.XPATH, '//*[@id="main"]/section/div[1]/div[1]/div/div/div[2]/div[2]/h1').text
                        
                        # 강좌 게시일
                        class_update_date = driver.find_element(By.XPATH, '//*[@id="main"]/section/div[3]/div/div/div[1]/div/section[4]/div/span[1]').text
                        # 강의생 수
                        class_stunum = driver.find_element(By.XPATH, '//*[@id="main"]/section/div[1]/div[1]/div/div/div[2]/div[3]/span[2]/strong').text
                        # 평점
                        try :
                            class_grade = driver.find_element(By.XPATH, '//*[@id="main"]/section/div[1]/div[1]/div/div/div[2]/div[3]/span[1]/strong').text
                        except :
                            class_grade = '0'
                            pass

                        # 난이도
                        class_level = driver.find_element(By.XPATH, '//*[@id="description"]/h2[1]/strong[1]').text

                        # 강의 총 시간
                        if "D" in driver.find_element(By.XPATH, '//*[@id="main"]/section/div[3]/div/div/div[2]/aside/div[2]/div/div[1]').text :
                            class_time = driver.find_element(By.XPATH, '//*[@id="main"]/section/div[3]/div/div/div[2]/aside/div[2]/div[1]/div[3]/div/div[2]').text

                        elif "신규" in driver.find_element(By.XPATH, '//*[@id="main"]/section/div[3]/div/div/div[2]/aside/div[2]/div/div[1]').text:
                            class_time = driver.find_element(By.XPATH, '//*[@id="main"]/section/div[3]/div/div/div[2]/aside/div[2]/div[1]/div[3]/div/div[2]').text

                        elif "할인" in driver.find_element(By.XPATH, '//*[@id="main"]/section/div[3]/div/div/div[2]/aside/div[2]/div/div[1]').text:
                            class_time = driver.find_element(By.XPATH, '//*[@id="main"]/section/div[3]/div/div/div[2]/aside/div[2]/div[1]/div[3]/div/div[2]').text

                        else :
                            class_time = driver.find_element(By.XPATH, '//*[@id="main"]/section/div[3]/div/div/div[2]/aside/div[2]/div[1]/div[2]/div/div[2]').text


                        list_data.append([class_name,class_stunum, class_price, class_grade, class_level, class_time, class_update_date])

                        driver.back()

                        WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, '//*[@id="courses_section"]/div/div/div/main/div[4]/div/div[1]/div/a/div[2]/div[1]')))
                except NoSuchElementException as e:
                    break
            


In [11]:

# 정의한 함수를 불러와 크롤링을 시작합니다.
for i in want_list:
    driver.get(f'https://www.inflearn.com/courses/{i}?order=seq')
    inflearn_crawling(i)
    df = pd.DataFrame(list_data)
    df.to_csv(f'{i}카테고리 데이터.csv', index=False)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.217)
Stacktrace:
	GetHandleVerifier [0x00007FF68EEB2142+3514994]
	(No symbol) [0x00007FF68EAD0CE2]
	(No symbol) [0x00007FF68E9776AA]
	(No symbol) [0x00007FF68E950AFD]
	(No symbol) [0x00007FF68E9ECB1B]
	(No symbol) [0x00007FF68EA0218F]
	(No symbol) [0x00007FF68E9E5D93]
	(No symbol) [0x00007FF68E9B4BDC]
	(No symbol) [0x00007FF68E9B5C64]
	GetHandleVerifier [0x00007FF68EEDE16B+3695259]
	GetHandleVerifier [0x00007FF68EF36737+4057191]
	GetHandleVerifier [0x00007FF68EF2E4E3+4023827]
	GetHandleVerifier [0x00007FF68EC004F9+689705]
	(No symbol) [0x00007FF68EADC048]
	(No symbol) [0x00007FF68EAD8044]
	(No symbol) [0x00007FF68EAD81C9]
	(No symbol) [0x00007FF68EAC88C4]
	BaseThreadInitThunk [0x00007FFC7131257D+29]
	RtlUserThreadStart [0x00007FFC7240AA58+40]


In [12]:
df = pd.DataFrame(list_data)
df

,0,1,2,3,4,5,6
0,스프링 핵심 원리 - 기본편\n대시보드,"30,472명","₩88,000\n₩61,600",(5.0),초급자,총 65개 수업 (12시간 5분),None
1,스프링 부트 - 핵심 원리와 활용\n대시보드,"8,579명","₩99,000\n₩69,300",(5.0),초급자,총 107개 수업 (15시간 45분),None
2,스프링 핵심 원리 - 고급편\n대시보드,"15,179명","₩121,000\n₩84,700",(5.0),중급자,총 125개 수업 (16시간 44분),None
3,김영한의 실전 자바 - 기본편\n대시보드,"5,092명","₩44,000\n₩30,800",(5.0),초급자,총 98개 수업 (16시간 51분),None
4,모든 개발자를 위한 HTTP 웹 기본 지식\n대시보드,"27,551명","₩44,000\n₩30,800",(5.0),초급자,총 41개 수업 (5시간 40분),None
...,...,...,...,...,...,...,...
3509,스프링 부트 - 핵심 원리와 활용\n대시보드,"8,582명","₩99,000\n₩69,300",(5.0),초급자,총 107개 수업 (15시간 45분),강의 게시일 : 2023년 02월 28일
3510,스프링 핵심 원리 - 고급편\n대시보드,"15,183명","₩121,000\n₩84,700",(5.0),중급자,총 125개 수업 (16시간 44분),강의 게시일 : 2021년 10월 27일
3511,김영한의 실전 자바 - 기본편\n대시보드,"5,102명","₩44,000\n₩30,800",(5.0),초급자,총 98개 수업 (16시간 51분),강의 게시일 : 2023년 11월 28일
3512,모든 개발자를 위한 HTTP 웹 기본 지식\n대시보드,"27,555명","₩44,000\n₩30,800",(5.0),초급자,총 41개 수업 (5시간 40분),강의 게시일 : 2020년 12월 20일
